In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Read chat to DF

In [ ]:
data = pd.read_json('../Data/dva_dimona_vovonjaka.json')

In [ ]:

allChats = json_normalize(data.chats[1])

In [ ]:
chat = json_normalize(allChats.messages[0])
chat.info()

DF cleanup

In [ ]:
chat = chat[chat['type']=='message']
chat = chat[['id', 'date', 'edited', 'text', 'from', 'media_type', 'sticker_emoji', 'reply_to_message_id']]
chat['date'] = chat['date'].apply(lambda x: pd.Timestamp(x))
chat['edited'] = chat['edited'].apply(lambda x: pd.Timestamp(x))
chat.replace(pd.Timestamp('1970-01-01T01:00:00'), pd.NA, inplace=True)
chat.info()

Statistics about message types

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize = (12,4))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

chat['from'].value_counts().plot(kind='bar', title='Количество сообщений', ax=axes[0])
chat[chat.media_type == 'sticker']['from'].value_counts().plot(kind='bar', title='Количество стикеров', ax=axes[1])
chat[chat.text.apply(lambda x: not isinstance(x, str))]['from'].value_counts().plot(kind='bar', title='Количество ссылок', ax=axes[2])
chat[chat.media_type == 'video_file']['from'].value_counts().plot(kind='bar', title='Количество видео', ax=axes[3])

Extract data from non-text messages

In [ ]:
msgs = np.array([])
mentions = np.array([])
links = np.array([])
for l in chat[chat.text.apply(lambda x: isinstance(x, list))].text.values:
    for item in l:
        if isinstance(item, str):
            msgs = np.append(item, msgs)
        if isinstance(item, dict):
            if item['type'] == 'link':
                links = np.append(item['text'], links)
            if item['type'] == 'mention':
                mentions = np.append(item['text'], mentions)
                # mentions.append(item['text'])

print('Messages: ', msgs)
print()
print('Mentions: ', mentions)
print()
print('Links: ', links)

### Define tokenixzation function

In [ ]:
import nltk
from pymystem3 import Mystem
from string import punctuation

nltk.download('stopwords')
nltk.download('punkt')

mystem = Mystem() 
def tokenize(text:str, stopWords: []):
    tokens = mystem.lemmatize(text.lower())
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]

    for p in punctuation:
        if p != '-':
            tokens = [t.replace(p, " ") for t in tokens]

    return [token.strip() for token in tokens if token.strip() not in stopWords\
              and token.strip() != "" \
              and not token.isdigit()
              and token.strip() >= 'А'
              and token.strip() <='я'
              and token.strip() not in punctuation]

Define stop-words

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...', '✔', '•', '’', 'че', 'ток', 'шо', 'тип'])

Difine function for getting user vocabulary

In [ ]:
from collections import Counter

def getTextMessage(obj):
    if isinstance(obj, str):
        return obj
    elif isinstance(obj, list):
        text = ''
        for item in obj:
            if isinstance(item, str):
                text += item.strip() + ' '
        return text
    else:
        return pd.NA

def getMessageChat(df):
    messageChat = df[['text', 'from', 'date']]
    messageChat.text = messageChat.text.apply(getTextMessage)
    return messageChat

def getTextMessages(data):
    messages = data[data.apply(lambda x: isinstance(x, str))]
    nonMessages = data[data.apply(lambda x: isinstance(x, list))]
    msgs = []
    for l in nonMessages.values:
        for item in l:
            if isinstance(item, str):
                msgs.append(item)
    return np.append(messages.values, msgs)

def getSentences(data):
    sentences = []
    for msg in getTextMessages(data):
        if msg >= 'А':
            sentences.extend(map(lambda s: s.capitalize().strip() ,nltk.sent_tokenize(msg)))
    return np.array(sentences)

def getTokens(data, stopWords=[]):
    return tokenize(' '.join(getTextMessages(data)), stopWords)

def getTokensFrequency(data, stopWords=[]):
    tokens = getTokens(data, stopWords)
    return Counter(tokens)

def getUserVocabulary(sender:str, stopWords=[]):
    allMsgs = chat[chat['from'] == sender].text
    tokens = getTokens(allMsgs, stopWords)
    return Counter(tokens)

### Get user most used words statistic 

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize = (15,4))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

z_words, z_counts = zip(*getUserVocabulary('Dez Dezsson').most_common(20))
d_words, d_counts = zip(*getUserVocabulary('Дима Лацюга').most_common(20))
v_words, w_counts = zip(*getUserVocabulary('Вова Свинухов').most_common(20))

axes[0].set_title('Dez Dezsson')
axes[0].barh(z_words, z_counts)

axes[1].set_title('Дима Лацюга')
axes[1].barh(d_words, d_counts)

axes[2].set_title('Вова Свинухов')
axes[2].barh(v_words, w_counts)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize = (15,4))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

z_words, z_counts = zip(*getUserVocabulary('Dez Dezsson', stop_words).most_common(20))
d_words, d_counts = zip(*getUserVocabulary('Дима Лацюга', stop_words).most_common(20))
v_words, w_counts = zip(*getUserVocabulary('Вова Свинухов', stop_words).most_common(20))

axes[0].set_title('Dez Dezsson')
axes[0].barh(z_words, z_counts)

axes[1].set_title('Дима Лацюга')
axes[1].barh(d_words, d_counts)

axes[2].set_title('Вова Свинухов')
axes[2].barh(v_words, w_counts)

In [ ]:
z_len=len(getUserVocabulary('Dez Dezsson', stop_words).keys())
d_len=len(getUserVocabulary('Дима Лацюга', stop_words).keys())
v_len=len(getUserVocabulary('Вова Свинухов', stop_words).keys())
plt.bar(['Dez Dezsson', 'Дима Лацюга', 'Вова Свинухов'], [z_len, d_len, v_len])
plt.title("Словарный запас")

In [ ]:
z_count = len(chat[chat['from'] == 'Dez Dezsson'])
d_count = len(chat[chat['from'] == 'Дима Лацюга'])
v_count = len(chat[chat['from'] == 'Вова Свинухов'])

print(z_count, d_count, v_count)

plt.bar(['Dez Dezsson', 'Дима Лацюга', 'Вова Свинухов'], [z_len/z_count, d_len/d_count, v_len/v_count])
plt.title("Словарный запас vs колличество сообщений")

## Find n-gramms

In [ ]:
def findNGramms(data, n=1, most_common=20, stop_words=[]):
    tokens = getTokens(data, stop_words)
    words=nltk.ngrams(tokens, n)
    words=nltk.FreqDist(words)
    return words

### N-граммы Димона

In [ ]:
findNGramms(chat[chat['from'] == 'Дима Лацюга'].text, n=2, stop_words=stop_words).plot(20, title = 'Дима Лацюга 2-gramms')

In [ ]:
findNGramms(chat[chat['from'] == 'Дима Лацюга'].text, n=3, stop_words=stop_words).plot(20, title = 'Дима Лацюга 3-gramms')

In [ ]:
findNGramms(chat[chat['from'] == 'Дима Лацюга'].text, n=4, stop_words=stop_words).plot(20, title = 'Дима Лацюга 4-gramms')

### N-граммы Вовоняки

In [ ]:
findNGramms(chat[chat['from'] == 'Вова Свинухов'].text, n=2, stop_words=stop_words).plot(20, title = 'Вова Свинухов 2-gramms')

In [ ]:
findNGramms(chat[chat['from'] == 'Вова Свинухов'].text, n=3, stop_words=stop_words).plot(20, title = 'Вова Свинухов 3-gramms')

In [ ]:
findNGramms(chat[chat['from'] == 'Вова Свинухов'].text, n=4, stop_words=stop_words).plot(20, title = 'Вова Свинухов 4-gramms')

### Мои N-граммы

In [ ]:
findNGramms(chat[chat['from'] == 'Dez Dezsson'].text, n=2, stop_words=stop_words).plot(20, title = 'Dez Dezsson 2-gramms')

In [ ]:
findNGramms(chat.text, n=2, stop_words=stop_words).plot(20, title = 'Наиболее частые 2-gramms')

# Text generation

In [ ]:
import markovify

def generateText(data, n=10):
    text = ". ".join(getSentences(data))
    textModel = markovify.Text(text, state_size=3, well_formed = True)
    text = ''
    for i in range(n):
        text += textModel.make_short_sentence(480, tries=100) + ' '
    return text
    
generateText(chat.text, 5)

## NaiveBayes classifier

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(train.text.values)

classifier = MultinomialNB()
classifier.fit(counts, train['from'].values)

examples = ['Бля посоны', "Павленский из той же тусовки, почитай про их перфомансы)", "Да и так вроде заебись, или бывает ошибается?", "если нет юзер специфик слов, то обычно выдает Димона, потому шо у него тупо сообщений больше"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

In [ ]:
from sklearn.model_selection import train_test_split

def predictSender(train_df, msgs:list):
    cv = CountVectorizer()
    clsf = MultinomialNB()
    
    message_counts = cv.fit_transform(train_df.text.values)
    clsf.fit(message_counts, train_df['from'].values)
    pr = clsf.predict(cv.transform(msgs))
    return  pd.DataFrame({'text': msgs, 'from': pr})

def get_test_results(train_df, test_df):
    prediction = predictSender(train_df, test_df.text)
    messages, real, predicted = [], [], []
    for index, row in test_df.iterrows():
        p, r = prediction['from'][index], row['from']
        if r != p:
            messages.append(row['text'])
            real.append(r)
            predicted.append(p)
    return pd.DataFrame({'text': messages, 'real': real, 'predicted': predicted})

chat = getMessageChat(chat)
train, test = train_test_split(chat, test_size=0.2)
p = get_test_results(train, test)

print(f"Truth coefficient is: {len(p)/len(test):.4f}")
p[p.predicted != 'Дима Лацюга'].head(20)

In [ ]:
examples = ['Бля посоны', "Павленский из той же тусовки, почитай про их перфомансы)", "Да и так вроде заебись, или бывает ошибается?", "если нет юзер специфик слов, то обычно выдает Димона, потому шо у него тупо сообщений больше"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions